Documentation home: https://github.com/engrit-illinois/misc-handy-powershell-examples
By mseng3
Converted to a Jupyter Notebook by han44

To use this, you'll need a Jupyter Notebook editor. An easy one to get up and working is VSCode:
1. Open Powershell as admin
2. Run `winget install Microsoft.DotNet.SDK.8`
3. Download/Install VSCode e.g. `winget install Microsoft.VisualStudioCode`
4. Install the Polyglot Notebooks extension in Visual Studio Code

Additionally, some of the examples below only really make sense in an interactive Powershell 7 session, so make sure to install Powershell 7!
1. Open Powershell as admin
2. Run `winget install Microsoft.Powershell`
3. Open the new Powershell 7 terminal! Or even better, use the `Terminal` app

Additionally, some examples that interact with MECM can *only* be done in a native Powershell session, so don't run those here!

------------

Do something remotely on multiple computers, in parallel...
https://devblogs.microsoft.com/powershell/powershell-foreach-object-parallel-feature/
Note the parallel functionality of ForEach-Object requires PowerShell 7
To do them NOT in parallel, just remove the `-ThrottleLimit` and `-Parallel` parameters from `ForEach-Object`

In [ ]:
# ...based on specific given computer names
$comps = "MEL-1001-10","KH-105-03","KH-107-03","EH-406B8-28" # etc., etc.
$comps

In [ ]:
# ...based on an AD name query
$comps = Get-ADComputer -Filter { Name -like "gelib-4c-*" } | Select -First 10 # to get the first 10 like-named computers
$comps

In [ ]:
# ...based on one or more AD name queries
$queries = "gelib-4e-*","dcl-l426-*","mel-1001-01"
$searchbase="OU=Engineering,OU=Urbana,DC=ad,DC=uillinois,DC=edu"
$comps = @()
foreach($query in @($queries)) {
	$results = Get-ADComputer -SearchBase $searchbase -Filter "name -like `"$query`"" -Properties *
	$comps += @($results)
}
# Return just the name, for the sake of this example
$comps.Name

In [ ]:
# ...based on direct membership rules of an MECM collection
Prep-MECM
$comps = Get-CMCollectionDirectMembershipRule -CollectionName "UIUC-ENGR-IS mseng3 Test VMs (direct membership)" | Select -ExpandProperty RuleName | Sort

In [ ]:
# ...based on sequentially-named computers (i.e. a lab)
$lab = "ECEB-3024"
$nums = @(4,5,7,11,14)
$comps = @()
$nums | ForEach-Object {
    $num = ([string]$_).PadLeft(2,"0")
    $comps += "$lab-$($num)"
}
$comps

In [ ]:
# Doing the thing
# if the members of $comps are AD or otherwise PowerShell object, instead of just strings
if($comps.ObjectClass -eq 'computer'){
    $compNames = $comps | Select -ExpandProperty "Name" 
}
$compNames | ForEach-Object -ThrottleLimit 15 -Parallel {
    Write-Host "Processing $_..."
    if(Test-Connection $_ -Quiet -Count 1){                 # Adding a ping test to avoid timeout
        Invoke-Command -ComputerName $_ -ScriptBlock {
            # Do stuff here
        }
    }
}

--------------

Start a new, elevated PowerShell process in the current directory:
Useful because Windows does not allow Powershell to  elevate an existing session
https://superuser.com/a/1256947/137753

In [ ]:
# Powershell 5
Start-Process powershell -Verb runas -ArgumentList "-NoExit -c cd '$pwd'"

In [ ]:
# Powershell 7
Start-Process pwsh -Verb runas -ArgumentList "-NoExit -c cd '$pwd'"

------------------

Create a new shortcut

In [ ]:
$pathFolder = "$env:userprofile\Desktop"
$pathLnk = "$pathFolder\calc.lnk"
$pathTarget = "C:\Windows\System32\calc.exe"

$shell = New-Object -ComObject WScript.Shell
$shortcut = $shell.CreateShortcut($pathLnk)
$shortcut.TargetPath = $pathTarget
$shortcut.Save()

# Open that folder
Invoke-Item -Path $pathFolder

# Open the calculator via the shortcut
Invoke-Item -Path $pathLnk

---------------------

Modify the target field of all existing shortcuts in the current directory

Intentionally keeping this as markdown since it can be destructive.

```powershell
$lnks = dir "*.lnk"
$lnks | ForEach-Object {
    # Create a temporary shortcut object to work with
    $lnk = (New-Object -ComObject 'WScript.Shell').CreateShortCut($_.FullName)
    # Modifies the base target file path
    $lnk.TargetPath = $lnk.TargetPath.Replace("appv1.exe","appv2.exe")
    # Modifies the arguments given to the target file
    $lnk.Arguments = $lnk.Arguments.Replace("-Param1 `"Hello World!`"","Param2 `"Hello.`"")
    # Modifies the "Start in" path
    $lnk.WorkingDirectory = $lnk.WorkingDirectory.Replace("c:\program files\","d:\test\")
    # Apply the changes to the actual shortcut file
    $lnk.Save()
}
```

--------------------

Create large dummy files

https://www.windows-commandline.com/how-to-create-large-dummy-file/

In [ ]:
fsutil file createnew $env:userprofile\bigtestfile.txt 1000 # 1KB

# Open that folder
Invoke-Item -Path "$env:userprofile"

-------------------------

Find 10 largest files

https://social.technet.microsoft.com/Forums/ie/en-US/838ed753-2bcf-49b8-9321-775c5ef12f13/finding-largest-files?forum=winserverpowershell

In [1]:
Get-ChildItem "$env:USERPROFILE" -Recurse -File | 
    Sort "length" -Descending | 
        Select @{Name="Size";Expression={"$([math]::Round(($_.length / 1MB),2))MB"}},"FullName","CreationTime","LastWriteTime","LastAccessTime" -First 10


Size           : 16665.65MB
FullName       : C:\Users\han44\OneDrive - University of Illinois - Urbana\Documents - Engineering 
                 IT Shared Services\[IT Ops] Siebel 
                 Pod\glen_test\teams_upload_test\nvidia_hpc_sdk_test_for_upload.tar
CreationTime   : 12/19/2023 2:46:46 PM
LastWriteTime  : 12/12/2022 12:25:48 PM
LastAccessTime : 12/19/2023 2:46:46 PM

Size           : 16665.65MB
FullName       : C:\Users\han44\OneDrive - University of Illinois - Urbana\Documents - Engineering 
                 IT Shared Services\[IT Ops] Siebel 
                 Pod\glen_test\teams_upload_test\nvidia_hpc_sdk_test_for_upload_3.tar
CreationTime   : 12/19/2023 2:46:46 PM
LastWriteTime  : 12/12/2022 12:25:48 PM
LastAccessTime : 12/19/2023 2:46:46 PM

Size           : 16665.65MB
FullName       : C:\Users\han44\OneDrive - University of Illinois - Urbana\Documents - Engineering 
                 IT Shared Services\[IT Ops] Siebel 
                 Pod\glen_test\teams_upload_test

The same thing but formatted to a table output

In [ ]:
Get-ChildItem "$env:USERPROFILE" -Recurse -File | 
    Sort "length" -Descending | 
        Select @{Name="Size";Expression={"$([math]::Round(($_.length / 1MB),2))MB"}},"FullName","CreationTime","LastWriteTime","LastAccessTime" -First 10 |
            Format-Table

---------------------------------

Find the largest user profiles greater than 1GB across multiple computers

In [ ]:
$lab = 'eceb-5080'
$comps = Get-ADComputer -Filter "name -like '$lab*'" -SearchBase "OU=Instructional,OU=Desktops,OU=Engineering,OU=Urbana,DC=ad,DC=uillinois,DC=edu"
$data = $comps | ForEach-Object {
	$comp = $_.Name
    if(Test-Connection -TargetName $comp -Quiet -Count 1){
        Invoke-Command -ComputerName $comp -ScriptBlock {
            Get-Item -Path "c:\users\*" | ForEach-Object {
                Get-ChildItem $_ -Recurse -File | Measure-Object -Property length -Sum -Maximum | Add-Member -NotePropertyName "User" -NotePropertyValue $_.Name -PassThru
            }
        }
    }
} | Select PSComputerName,User,Count,@{N="Max (GB)";E={[int]($_.Maximum/1GB)}},@{N="Sum (GB)";E={[int]($_.Sum/1GB)}}
$data = $data | Where { $_."Sum (GB)" -ge 1 }
$data | Sort PSComputerName,@{Expression="Sum (GB)";Descending=$true} | Format-Table

The same thing, parallelized!

In [ ]:
$lab = 'eceb-5080'
$comps = Get-ADComputer -Filter "name -like '$lab*'" -SearchBase "OU=Instructional,OU=Desktops,OU=Engineering,OU=Urbana,DC=ad,DC=uillinois,DC=edu"
$data = $comps | ForEach-Object -ThrottleLimit 20 -Parallel {
	$comp = $_.Name
    if(Test-Connection -TargetName $comp -Quiet -Count 1){
        Invoke-Command -ComputerName $comp -ScriptBlock {
            Get-Item -Path "c:\users\*" | ForEach-Object {
                Get-ChildItem $_ -Recurse -File | Measure-Object -Property length -Sum -Maximum | Add-Member -NotePropertyName "User" -NotePropertyValue $_.Name -PassThru
            }
        }
    }
} | Select PSComputerName,User,Count,@{N="Max (GB)";E={[int]($_.Maximum/1GB)}},@{N="Sum (GB)";E={[int]($_.Sum/1GB)}}
$data = $data | Where { $_."Sum (GB)" -ge 1 }
$data | Sort PSComputerName,@{Expression="Sum (GB)";Descending=$true} | Format-Table